In [ ]:
!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.2/698.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.13.1
    Uninstalling aiohttp-3.13.1:
      Successfully uninstalled aiohttp-3.13.1


In [ ]:
import asyncio
import os
from aiogram import Bot, Dispatcher, types
from aiogram.client.default import DefaultBotProperties
from aiogram.filters import CommandStart
from aiogram.enums import ParseMode
from dotenv import find_dotenv, load_dotenv

from transformers import AutoTokenizer
import torch
import pickle


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Жалобы жкх/веса и бинарники/sber/sber_tokenizer')

try:
    with open('/content/drive/MyDrive/Жалобы жкх/веса и бинарники/sber/label_binarizers.pkl', 'rb') as f:
        binarizers = pickle.load(f)
    print("✅ Binarizers загружены!")
except FileNotFoundError:
    print("❌ Файл binarizers не найден!")
    binarizers = {
        'lb_main': type('obj', (object,), {'classes_': ['категория1', 'категория2']}),
        'lb_emotion': type('obj', (object,), {'classes_': ['эмоция1', 'эмоция2']}),
        'lb_urgency': type('obj', (object,), {'classes_': ['срочно', 'не срочно']})
    }

# Архмтектура модели
class MultiTaskBERT(torch.nn.Module):
    def __init__(self, model_name, num_main_labels, num_emotion_labels, num_urgency_labels):
        super().__init__()
        from transformers import AutoModel
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = torch.nn.Dropout(0.1)

        # Получаем размерность из модели
        hidden_size = self.bert.config.hidden_size

        self.classifier_main = torch.nn.Linear(hidden_size, num_main_labels)
        self.classifier_emotion = torch.nn.Linear(hidden_size, num_emotion_labels)
        self.classifier_urgency = torch.nn.Linear(hidden_size, num_urgency_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)

        main_logits = self.classifier_main(pooled_output)
        emotion_logits = self.classifier_emotion(pooled_output)
        urgency_logits = self.classifier_urgency(pooled_output)

        return main_logits, emotion_logits, urgency_logits

# Загрузка модели
model = MultiTaskBERT(
    model_name=model_name,
    num_main_labels=len(binarizers['lb_main'].classes_),
    num_emotion_labels=len(binarizers['lb_emotion'].classes_),
    num_urgency_labels=len(binarizers['lb_urgency'].classes_)
)

# Загрузка весов
model.load_state_dict(torch.load('/content/drive/MyDrive/Жалобы жкх/веса и бинарники/sber/multitask_model_weights.pth', map_location='cpu'))
model.eval()
print("✅ Модель загружена!")


✅ Binarizers загружены!


pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

✅ Модель загружена!


In [ ]:
from google.colab import userdata

In [ ]:
TOKEN = userdata.get('TOKEN')

In [ ]:

import nest_asyncio
nest_asyncio.apply()

ALLOWED_UPDATES = ['message, edited_message']

bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher()


def predict_complaint(text):
    """Предсказывает категорию, эмоцию и срочность для текста жалобы"""

    # Токенизируем текст
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

    # Предсказываем
    with torch.no_grad():
        main_logits, emotion_logits, urgency_logits = model(
            inputs['input_ids'],
            inputs['attention_mask']
        )

    # Преобразуем логиты в текстовые метки
    main_pred = torch.argmax(main_logits, dim=1).item()
    emotion_pred = torch.argmax(emotion_logits, dim=1).item()
    urgency_pred = torch.argmax(urgency_logits, dim=1).item()

    # Используем binarizers для обратного преобразования
    try:
        main_label = binarizers['lb_main'].classes_[main_pred]
        emotion_label = binarizers['lb_emotion'].classes_[emotion_pred]
        urgency_label = binarizers['lb_urgency'].classes_[urgency_pred]
    except (IndexError, AttributeError):
        # Если binarizers не работают, используем числовые значения
        main_label = f"Категория_{main_pred}"
        emotion_label = f"Эмоция_{emotion_pred}"
        urgency_label = f"Срочность_{urgency_pred}"

    return {
        'category': main_label,
        'emotion': emotion_label,
        'urgency': urgency_label
    }
#------------------БОТ------------------------------

@dp.message(CommandStart())
async def send_welcome(message: types.Message):
    await message.answer(
        "👋 Привет! Я бот для обработки жалоб ЖКХ.\n"
        "Просто опишите свою проблему, и я её классифицирую!\n\n"
        "Пример: 'В подъезде не горит свет, очень страшно'"
    )

@dp.message()
async def handle_complaint(message: types.Message):
    user_text = message.text
    thinking_msg = await message.answer("🔍 Анализирую вашу жалобу...")

    try:
        prediction = predict_complaint(user_text)
        response = (
            f"📋 **Результат анализа:**\n\n"
            f"🏷️ **Категория:** {prediction['category']}\n"
            f"😊 **Эмоция:** {prediction['emotion']}\n"
            f"⚡ **Срочность:** {prediction['urgency']}\n\n"
            f"💡 **Рекомендация:** {get_advice(prediction)}"
        )

        await thinking_msg.delete()
        await message.answer(response, parse_mode='Markdown')

    except Exception as e:
        await thinking_msg.delete()
        await message.answer(f"❌ Ошибка: {str(e)}")

def get_advice(prediction):
    """Генерирует простой совет на основе предсказания"""
    advice_templates = {
        'высокая': "Рекомендуем срочно обратиться в аварийную службу!",
        'критическая': "🚨 НЕМЕДЛЕННО вызывайте аварийную службу!",
        'средняя': "Обратитесь в управляющую компанию в рабочее время.",
        'низкая': "Проблема будет решена в плановом порядке."
    }

    return advice_templates.get(prediction['urgency'],
                               "Специалист свяжется с вами в ближайшее время.")


async def main():
    print("🤖 Бот запускается...")
    await bot.delete_webhook(drop_pending_updates=True)
    await dp.start_polling(bot, allowed_updates=dp.resolve_used_update_types())

asyncio.run(main())

🤖 Бот запускается...
